In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [2]:
cd Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


# **Deployement of the LSTM model**

In [0]:
# import the libraries
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import joblib

In [0]:
# Load the model, the scaler function and the testing sample
files_model=load_model('best_lstm1.h5')
files_scaler=joblib.load("lstm_scaler.pkl")
files=pd.read_csv('sample.csv')

# Auxillary functions
def reshapes(x):
  return x.reshape(-1,1,6)

def dummies_transform(x):
  """One Hot encode the File Types and concantenate the output to the original dataset"""
  ftype=pd.get_dummies(x['File Type'], drop_first=True)
  dataset=pd.concat([x, ftype], axis = 1)
  dataset=dataset.drop('File Type', axis = 1)
  return dataset

def classification(x):
  results=[]
  for i in range(len(x)):
    if x[i][0]<0.5:
      results.append('Cold') 
    else:
      results.append('Hot') 
  return results

In [5]:
files

,File Type,Quality,Access Requests,Usage Requests,Status
0,L,0,11,61,0
1,B,1,2,40,1
2,S,1,27,1,1
3,B,1,0,0,0
4,B,1,2,88,1
5,L,0,0,0,0
6,C,1,15,30,0


In [0]:
# Preprocess the data to fit the format of the training data
data=dummies_transform(files)

y=data['Status']
x=data.drop('Status', axis=1)

X=files_scaler.fit_transform(x)
X=X.reshape(-1,1,6)

# Use the model to predict whether the files are hot or cold 
pred=files_model.predict(X)

#Assign the file state based on the probability obtained from pred
predictions=classification(pred)


#**Management of the storages**

Let's assume that all the files are stored as generated for the testing dataset below. We want to move them according to based on their location status and their state prediction.

In [0]:
def manager(data):
  """It takes the model outputs and distribute them according to the status of the file"""
  for i in data:
    if i [2]== 'Hot' and i[1] == 'Storage':  #File is hot and needs to be moved to dCache for faster access
      print(f'{i[0]} has been moved to the dCache and is ready for use')
    elif i [2]== 'Hot' and i[1] != 'Storage': #!Storage means file is already in dCache therefore it's not necessary to take any action
      print(f'{i[0]} is already in dCache. Skipping this access request')
    elif i [2]== 'Cold' and i[1] == 'Storage': #The file has a state of cold and as such does not need to be moved to the Storage drive
      print(f'{i[0]} is already in Storage and does not need to be moved')
    else: # File is cold but was still in dCache therefore it must be moved to the Storage drive to free up disk space
      print(f'{i[0]} is cold and has been moved to Storage')

In [8]:
# creating a storage and a dCache
import random as rd
rd.seed(101)
filenames=['file_'+str(x) for x in range(7)]
location=[i for i in rd.choices(['Storage', 'dCache'],weights=[0.5,0.5], k=7)]

#creating an archives for the files
archives=list(zip(filenames, location, predictions))
archives 

[('file_0', 'dCache', 'Cold'),
 ('file_1', 'Storage', 'Hot'),
 ('file_2', 'dCache', 'Hot'),
 ('file_3', 'dCache', 'Cold'),
 ('file_4', 'Storage', 'Hot'),
 ('file_5', 'dCache', 'Cold'),
 ('file_6', 'Storage', 'Cold')]

In [10]:
#applying the manager
manager(archives)

file_0 is cold and has been moved to Storage
file_1 has been moved to the dCache and is ready for use
file_2 is already in dCache. Skipping this access request
file_3 is cold and has been moved to Storage
file_4 has been moved to the dCache and is ready for use
file_5 is cold and has been moved to Storage
file_6 is already in Storage and does not need to be moved
